# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Aiml/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'stance_detection.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_bodies_data = pd.read_csv(project_path + "train_bodies.csv")
train_stances_data = pd.read_csv(project_path + "train_stances.csv")

In [0]:
dataset = pd.merge(train_bodies_data, train_stances_data, on = 'Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [8]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [11]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
tokenizer.fit_on_texts(dataset.articleBody)
tokenizer.fit_on_texts(dataset.Headline)

In [14]:
len(tokenizer.word_counts)

27873

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = dataset.articleBody
articles = []
for text in texts:
  article = nltk.sent_tokenize(text)
  articles.append(article)

## Check 2:

first element of texts and articles should be as given below. 

In [16]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [17]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence

data =  np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH), dtype = int)

In [0]:
for index_one, article in enumerate(articles):
  article = article[0:MAX_SENTS]
  for index_two, sentence in enumerate(article):
    list_of_words = text_to_word_sequence(sentence)[0:MAX_SENT_LENGTH]
    encodings = []
    for word in list_of_words:
      encoding = tokenizer.word_index.get(word)
      encodings.append(encoding)
    if(len(encodings) < MAX_SENT_LENGTH):
      encodings.extend([0] * (MAX_SENT_LENGTH - len(encodings)))
    data[index_one][index_two] = encodings

### Check 3:

Accessing first element in data should give something like given below.

In [20]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = dataset.Headline
articles_heading = []
for text_heading in texts_heading:
  article_heading = nltk.sent_tokenize(text_heading)
  articles_heading.append(article_heading)

In [22]:
texts_heading[0]

'Soldier shot, Parliament locked down after gunfire erupts at war memorial'

In [23]:
articles_heading[0]

['Soldier shot, Parliament locked down after gunfire erupts at war memorial']

In [24]:
len(articles_heading)

49972

In [0]:
data_heading =  np.zeros((len(articles_heading), MAX_SENTS_HEADING, MAX_SENT_LENGTH), dtype = int)

In [0]:
for index_one, heading in enumerate(articles_heading):
  heading = heading[0:MAX_SENTS_HEADING]
  for index_two, head in enumerate(heading):
    list_of_words = text_to_word_sequence(head)[0:MAX_SENT_LENGTH]
    encodings = []
    for word in list_of_words:
      encoding = tokenizer.word_index.get(word)
      encodings.append(encoding)
    if(len(encodings) < MAX_SENT_LENGTH):
      encodings.extend([0] * (MAX_SENT_LENGTH - len(encodings)))
    data_heading[index_one][index_two] = encodings

In [27]:
data_heading[49971, :, :]

array([[6105, 6363, 2165, 2157, 1856, 1998,   14,  284,  145, 2379,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(train_stances_data.Stance)

In [29]:
labels.head()

,agree,disagree,discuss,unrelated
0,0,0,0,1
1,1,0,0,0
2,0,0,0,1
3,0,0,0,1
4,0,1,0,0


### Check 4:

The shape of data and labels shoould match the given below numbers.

In [30]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels.iloc[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split

In [36]:
int(len(data) * 0.8)

39977

In [0]:
x_train = data[0 : int(len(data) * 0.8) + 1, :, :]
x_val = data[int(len(data) * 0.8) + 1 : len(data), :, :]

x_heading_train = data_heading[0 : int(len(data_heading) * 0.8) + 1, :, :]
x_heading_val = data_heading[int(len(data_heading) * 0.8) + 1 : len(data_heading), :, :]

y_train = labels.iloc[0:int(len(data) * 0.8) + 1, :]
y_val = labels.iloc[int(len(data) * 0.8) + 1 : len(data), :]

In [38]:
x_val.shape

(9994, 20, 20)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [39]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [43]:
# load the whole embedding into memory
embeddings_index = dict()
f = open(project_path+'glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((MAX_NB_WORDS, 100))


for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
      if(i >= MAX_NB_WORDS):
        continue
      else:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.layers import LSTM
from keras  import callbacks
from keras import optimizers
from keras.models import load_model

### Model

In [45]:
batch_size = 2
model = Sequential()
model.add(LSTM(4, batch_input_shape=(2, 20, 20), stateful=True))
model.add(Dense(4))

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import Callback

checkpoint = ModelCheckpoint("init_model.h5", monitor='loss', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=1, mode='auto')

### Compile and fit the model

In [47]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (2, 4)                    400       
_________________________________________________________________
dense_1 (Dense)              (2, 4)                    20        
Total params: 420
Trainable params: 420
Non-trainable params: 0
_________________________________________________________________


## Build the same model with attention layers included for better performance (Optional)

In [49]:
model.fit(x_train, y_train, epochs = 3, batch_size = batch_size, shuffle=True, callbacks = [checkpoint, early])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/3





39978/39978 [==============================] - 691s 17ms/step - loss: 0.1107 - acc: 0.7263

Epoch 00001: loss improved from inf to 0.11072, saving model to init_model.h5
Epoch 2/3
39978/39978 [==============================] - 689s 17ms/step - loss: 0.1073 - acc: 0.7306

Epoch 00002: loss improved from 0.11072 to 0.10732, saving model to init_model.h5
Epoch 3/3
39978/39978 [==============================] - 699s 17ms/step - loss: 0.1073 - acc: 0.7306

Epoch 00003: loss did not improve from 0.10732


In [0]:
init_model = load_model("init_model.h5")

In [51]:
init_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (2, 4)                    400       
_________________________________________________________________
dense_1 (Dense)              (2, 4)                    20        
Total params: 420
Trainable params: 420
Non-trainable params: 0
_________________________________________________________________


In [0]:
import math
from sklearn.metrics import mean_squared_error

batch_size = 2

trainPredict = init_model.predict(x_train, batch_size=batch_size)
init_model.reset_states()

testPredict = init_model.predict(x_val, batch_size=batch_size)

In [54]:
trainPredict

array([[0.06777345, 0.01220544, 0.18378237, 0.73406625],
       [0.06581904, 0.0120573 , 0.18527544, 0.7348941 ],
       [0.06539444, 0.0125639 , 0.18527444, 0.734783  ],
       ...,
       [0.06575301, 0.01214866, 0.1853378 , 0.73473936],
       [0.06681565, 0.01226378, 0.18450648, 0.7342817 ],
       [0.06538963, 0.01261345, 0.18524857, 0.7347706 ]], dtype=float32)

In [55]:
results = init_model.evaluate(x_val, y_val, batch_size = batch_size)

9994/9994 [==============================] - 84s 8ms/step


In [56]:
print("Mean Squared Error Loss: ", results[0])
print("Model Accuracy on Validation/Test Data on Validation/Test Data: ", results[1]*100)

Mean Squared Error Loss:  0.10600584612632599
Model Accuracy on Validation/Test Data on Validation/Test Data:  73.43406043626176


## Fit the model and report the accuracy score for the model with attention layer (Optional)

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [0]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, 20, 20), stateful=True))
model.add(Attention(1))
model.add(Dense(4))

In [0]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [60]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (1, 4)                    400       
_________________________________________________________________
attention_1 (Attention)      (1, 4)                    8         
_________________________________________________________________
dense_2 (Dense)              (1, 4)                    20        
Total params: 428
Trainable params: 428
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_att = ModelCheckpoint("init_model_att.h5", monitor='loss', verbose=1, save_best_only=True, mode='auto')
early_att = EarlyStopping(monitor='loss', min_delta=0, patience=5, verbose=1, mode='auto')

In [62]:
model.fit(x_train, y_train, epochs = 1, batch_size = 1, callbacks = [checkpoint_att, early_att])

Epoch 1/1
39978/39978 [==============================] - 1380s 35ms/step - loss: 0.1104 - acc: 0.7277

Epoch 00001: loss improved from inf to 0.11038, saving model to init_model_att.h5


In [63]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (1, 4)                    400       
_________________________________________________________________
attention_1 (Attention)      (1, 4)                    8         
_________________________________________________________________
dense_2 (Dense)              (1, 4)                    20        
Total params: 428
Trainable params: 428
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.load_weights("init_model_att.h5")

In [65]:
results = model.evaluate(x_val, y_val, batch_size=1)

9994/9994 [==============================] - 169s 17ms/step


In [66]:
# Attention vs Sequential Model
print("Attention Model Mean Squared Error Loss on Validation/Test Data: ", results[0])
print("Attention Model Accuracy on Validation/Test Data: ", results[1]*100)

## Results are similar in both the models. 

Attention Model Mean Squared Error Loss on Validation/Test Data:  0.10619420259788391
Attention Model Accuracy on Validation/Test Data:  73.43406043626176
